In [ ]:
from google.cloud.storage import Client
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
from pathlib import Path

from glambie.util.timeseries_helpers import derivative_to_cumulative

import seaborn as sns
sns.set_theme(palette='muted', style='whitegrid', font_scale=1.2)

In [ ]:
DATA_TRANSFER_BUCKET_NAME = "glambie-submissions"
PROJECT_NAME = "glambie"
storage_client = Client(project=PROJECT_NAME)

In [ ]:
list_of_blobs_in_bucket = storage_client.list_blobs(DATA_TRANSFER_BUCKET_NAME)
downloaded_files = []

In [ ]:
for blob in list_of_blobs_in_bucket:
    if 'glambie_csvs_to_publish_v2.zip' in blob.name:
        downloaded_files.append(blob.name)
        destination_file_path = os.path.join(local_path, blob.name)
        Path(destination_file_path).parent.mkdir(parents=True, exist_ok=True)
        with open(destination_file_path, "wb") as output_file:
            blob.download_to_file(output_file, raw_download=False)

In [ ]:
temporal_option = 'hydrological_years'
data_directory = local_path + 'glambie_csvs_to_publish' + '/' + temporal_option

In [ ]:
list_of_csvs = sorted(glob.glob(os.path.join(data_directory, '*.csv')))

In [ ]:
# choose a region
region = 'arctic_canada_south'

In [ ]:
for filename in list_of_csvs:
    if region in filename:
        region_dataframe = pd.read_csv(filename)

In [ ]:
plt.subplots(1, 1, figsize=(12,8))

plt.plot(region_dataframe.end_dates, region_dataframe.consensus_mwe, linewidth=3, zorder=2, label='Consensus change')
plt.fill_between(region_dataframe.end_dates, region_dataframe.consensus_mwe - region_dataframe.consensus_mwe_errors, region_dataframe.consensus_mwe + region_dataframe.consensus_mwe_errors, alpha=0.2)
plt.hlines(0, region_dataframe.end_dates.values[0], region_dataframe.end_dates.values[-1], linestyle='dashed', color='purple')
plt.xlabel('Year')
plt.ylabel('Elevation Change (meters water equivalent)')
plt.title(transform_string(region) + ' - change in elevation between 2000 and 2024')

plt.legend()

In [ ]:
# Start with a simple mwe plot - explain that the following plot shows how many m of ice each region has lost over the last 24 years
cumulative_data_all_mwe = derivative_to_cumulative(region_dataframe.start_dates, region_dataframe.end_dates, region_dataframe.consensus_mwe, return_type='dataframe')
cumulative_errors_all_mwe = derivative_to_cumulative(region_dataframe.start_dates, region_dataframe.end_dates, region_dataframe.consensus_mwe_errors, return_type='dataframe', calculate_as_errors=True)

In [ ]:
plt.subplots(1, 1, figsize=(12,8))

plt.plot(cumulative_data_all_mwe.dates, cumulative_data_all_mwe.changes, linewidth=3, zorder=2, label='Consensus change')
plt.fill_between(cumulative_data_all_mwe.dates, cumulative_data_all_mwe.changes - cumulative_errors_all_mwe.errors, cumulative_data_all_mwe.changes + cumulative_errors_all_mwe.errors, alpha=0.2)#
plt.xlabel('Year')
plt.ylabel('Cumulative Change [meters water equivalent]')
plt.title(transform_string(region) + ' - ice loss between 2000 and 2024')

plt.legend()

In [ ]:
# convert to cumulative changes - Gt all
cumulative_data_all_gt = derivative_to_cumulative(region_dataframe.start_dates, region_dataframe.end_dates, region_dataframe.consensus_gt, return_type='dataframe')
cumulative_errors_all_gt = derivative_to_cumulative(region_dataframe.start_dates, region_dataframe.end_dates, region_dataframe.consensus_gt_errors, return_type='dataframe', calculate_as_errors=True)

In [ ]:
cumulative_data_altimetry_gt = derivative_to_cumulative(region_dataframe.start_dates, region_dataframe.end_dates, region_dataframe.altimetry_gt, return_type='dataframe')
cumulative_data_gravimetry_gt = derivative_to_cumulative(region_dataframe.start_dates, region_dataframe.end_dates, region_dataframe.gravimetry_gt, return_type='dataframe')
cumulative_data_demdiff_and_glaciological_gt = derivative_to_cumulative(region_dataframe.start_dates, region_dataframe.end_dates, region_dataframe.demdiff_and_glaciological_gt, return_type='dataframe')

In [ ]:
plt.subplots(1, 1, figsize=(12,8))

plt.plot(cumulative_data_all_gt.dates, cumulative_data_all_gt.changes, linewidth=3, zorder=2, label='Consensus change')
plt.fill_between(cumulative_data_all_gt.dates, cumulative_data_all_gt.changes - cumulative_errors_all_gt.errors, cumulative_data_all_gt.changes + cumulative_errors_all_gt.errors, alpha=0.2)#
plt.plot(cumulative_data_altimetry_gt.dates, cumulative_data_altimetry_gt.changes, linestyle='dashed', zorder=1, alpha=0.7, label='Altimetry')
plt.plot(cumulative_data_gravimetry_gt.dates, cumulative_data_gravimetry_gt.changes, linestyle='dashed', zorder=1, alpha=0.7, label='Gravimetry')
plt.plot(cumulative_data_demdiff_and_glaciological_gt.dates, cumulative_data_demdiff_and_glaciological_gt.changes, linestyle='dashed', zorder=1, alpha=0.7, label='DemDiff and Glaciological')
plt.xlabel('Year')
plt.ylabel('Cumulative Change [Gt]')
plt.title(transform_string(region) + ' - Gt of ice loss between 2000 and 2024')

plt.legend(loc='lower left')

In [ ]:
# Compare to another region?
comparison_region = 'arctic_canada_north'

In [ ]:
for filename in list_of_csvs:
    if comparison_region in filename:
        region_dataframe_comparison = pd.read_csv(filename)

In [ ]:
plt.subplots(1, 1, figsize=(12,8))

plt.plot(cumulative_data_all_gt.dates, cumulative_data_all_gt.changes, linewidth=3, zorder=2, label='Consensus change - ' + transform_string(region))
plt.fill_between(cumulative_data_all_gt.dates, cumulative_data_all_gt.changes - cumulative_errors_all_gt.errors, cumulative_data_all_gt.changes + cumulative_errors_all_gt.errors, alpha=0.2)

plt.plot(cumulative_data_all_gt_comparison.dates, cumulative_data_all_gt_comparison.changes, linewidth=3, zorder=2, label='Consensus change - ' + transform_string(comparison_region))
plt.fill_between(cumulative_data_all_gt_comparison.dates, cumulative_data_all_gt_comparison.changes - cumulative_errors_all_gt_comparison.errors, cumulative_data_all_gt_comparison.changes + cumulative_errors_all_gt_comparison.errors, alpha=0.2)

plt.xlabel('Year')
plt.ylabel('Cumulative Change [Gt]')
plt.title('Regional comparison - Gt of ice loss between 2000 and 2024')

plt.legend()